In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import torch
import torch
import sys
import os
print(os.getcwd())
module_path = os.path.abspath(os.path.join('/data2/david3684/2024_arithmetic'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import numpy as np
from src.eval import eval_single_dataset_with_prediction, eval_single_dataset
from src.main import save_scale_factors
from src.args import parse_arguments
from src.datasets.common import get_dataloader, maybe_dictionarize
from src.datasets.registry import get_dataset
from src.modeling import ImageEncoder, ImageClassifier
from src.task_vectors import TaskVector
from tqdm import tqdm
from copy import deepcopy
import matplotlib.pyplot as plt
from datetime import datetime
import pickle
import open_clip

/data2/david3684/2024_arithmetic/src


In [7]:
class Args:
    def __init__(self):
        self.model = 'ViT-L-14'
        self.tasks = ['DTD', 'SUN397']
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.task_scale_factors = None
        self.save = '/data2/david3684/2024_arithmetic/checkpoints/ViT-L-14'
        self.data_location = '/data2/david3684/data'
        self.eval_datasets = None
        self.train_dataset = None
        self.exp_name = None
        self.results_db = None
        self.batch_size = 128
        self.lr = 0.001
        self.wd = 0.1
        self.ls = 0.0
        self.warmup_length = 500
        self.epochs = 10
        self.load = None
        self.cache_dir = None
        self.openclip_cachedir = '/data2/david3684/.cache/open_clip'
        self.initial_rank_ratio = 1.0
        self.low_rank_mode = 'SoRA'
        self.pretrained_model = 'openai'
        self.scale_shared_weight = False
        self.num_test_samples = 2048
        self.no_shared_weight = False

args = Args()

In [8]:
# Load finetuned weight

model_1 = torch.load('/data2/david3684/2024_arithmetic/checkpoints/ViT-L-14/DTD/finetuned.pt').to(args.device)
model_2 = torch.load('/data2/david3684/2024_arithmetic/checkpoints/ViT-L-14/SUN397/finetuned.pt').to(args.device)

In [9]:
def transform_key(old_key):
    if old_key.startswith('shared.attn.layer') or old_key.startswith('clip_vit'):
        parts = old_key.split('.')
        layer_idx = parts[3]
        # print(layer_idx)
        sub_key = parts[4]
        if sub_key in ['q', 'k', 'v']:
            return f'model.visual.transformer.resblocks.{layer_idx}.attn.{sub_key}_weight'
        elif sub_key == 'out_proj':
            return f'model.visual.transformer.resblocks.{layer_idx}.attn.out_proj.weight'
        elif sub_key == 'c_fc' or sub_key == 'c_proj':
            return f'model.visual.transformer.resblocks.{layer_idx}.mlp.{sub_key}.weight'
    return old_key

In [10]:
def format_shared_weight(shared_weight_state_dict, open_clip_state_dict_template):
    qkv_store = {}
    for old_key, value in shared_weight_state_dict.items():
        if 'diff' in old_key or 'scale_dict' in old_key:
            continue

        new_key = transform_key(old_key)
        layer_idx = new_key.split('.')[4]

        if layer_idx not in qkv_store:
            qkv_store[layer_idx] = {'q': None, 'k': None, 'v': None}

        weight_type = new_key.split('.')[-1]
        # in_proj.weight (q, k, v)
        if weight_type in ['q_weight', 'k_weight', 'v_weight']:
            qkv_store[layer_idx][weight_type[0]] = value
        else:  # out_proj.weight, c_fc.weight, c_proj.weight
            assert new_key in open_clip_state_dict_template
            open_clip_state_dict_template[new_key] = value

    for layer_idx, qkv in qkv_store.items():
        if all(v.bool().all().item() for v in qkv.values()):
            in_proj_weight = torch.cat([qkv['q'], qkv['k'], qkv['v']], dim=0)
            # concat qkv into 3072*1024 tensor
            new_key = f'model.visual.transformer.resblocks.{layer_idx}.attn.in_proj_weight'
            assert new_key in open_clip_state_dict_template
            open_clip_state_dict_template[new_key] = in_proj_weight
        else:
            print(
                f"Missing q, k, or v for layer {layer_idx}. q: {qkv['q']}, k: {qkv['k']}, v: {qkv['v']}")

    return open_clip_state_dict_template

#나머지 처리 필요

In [11]:
# format shaed weight into openclip state dict
shared_weight_state_dict = torch.load('/data2/david3684/2024_arithmetic/shared_weight/20241010_vanilla/rankmin_config_20241010_uni_vanilla_2_v0.bin')
zero_shot_encoder = ImageEncoder(args, keep_lang=False)

formatted_shared_weight = format_shared_weight(shared_weight_state_dict, zero_shot_encoder.state_dict())

print(zero_shot_encoder.load_state_dict(formatted_shared_weight))
zero_shot_encoder = zero_shot_encoder.to(args.device)

Loading ViT-L-14 pre-trained weights.
<All keys matched successfully>


In [17]:
# loop for task vector rank

for initial_rank_ratio in [0.1, 0.01, 0.001]:
    args.initial_rank_ratio = initial_rank_ratio
    print(f'Initial rank ratio: {initial_rank_ratio}')
    low_rank_task_vectors = {}
    
    # Build low rank task vectors
    for task in args.tasks:
        model = model_1 if task == 'DTD' else model_2
        # eval_single_dataset(model, task, args)
        finetuned_state_dict = model_1.state_dict() if task == 'DTD' else model_2.state_dict()
        low_rank_task_vectors[task] = TaskVector(args, zero_shot_encoder.state_dict(), finetuned_state_dict, task).to(args.device)
    
    low_rank_task_vector_sum = sum(low_rank_task_vectors.values())
    
    for task in args.tasks:
        print(f'Evaluate {task}')
        # Evaluate sinlge task model
        print('Single task model')
        low_rank_single_task_encoder = low_rank_task_vectors[task].apply_to(deepcopy(zero_shot_encoder), scaling_coef=1.0)
        eval_single_dataset(low_rank_single_task_encoder, task, args)
        
        # Evaluate multi task model
        print('Multi task model')
        low_rank_multi_task_encoder = low_rank_task_vector_sum.apply_to(deepcopy(zero_shot_encoder), scaling_coef=1.0)
        eval_single_dataset(low_rank_multi_task_encoder, task, args)

Initial rank ratio: 0.1
Building task vector with shared weight
Building task vector with shared weight
Evaluate DTD
Single task model
Classification head for ViT-L-14 on DTD exists at /data2/david3684/2024_arithmetic/checkpoints/ViT-L-14/head_DTD_openai.pt
Loading classification head from /data2/david3684/2024_arithmetic/checkpoints/ViT-L-14/head_DTD_openai.pt
Number of classes: 47
Train dataset size: 1880
Test dataset size: 1880


100%|██████████| 15/15 [00:29<00:00,  1.99s/it]


Done evaluating on DTD. Accuracy: 98.14%
Multi task model
Classification head for ViT-L-14 on DTD exists at /data2/david3684/2024_arithmetic/checkpoints/ViT-L-14/head_DTD_openai.pt
Loading classification head from /data2/david3684/2024_arithmetic/checkpoints/ViT-L-14/head_DTD_openai.pt
Number of classes: 47
Train dataset size: 1880
Test dataset size: 1880


100%|██████████| 15/15 [00:31<00:00,  2.12s/it]


Done evaluating on DTD. Accuracy: 96.33%
Evaluate SUN397
Single task model
Classification head for ViT-L-14 on SUN397 exists at /data2/david3684/2024_arithmetic/checkpoints/ViT-L-14/head_SUN397_openai.pt
Loading classification head from /data2/david3684/2024_arithmetic/checkpoints/ViT-L-14/head_SUN397_openai.pt
Number of classes: 397
Train dataset size: 87003
Test dataset size: 21751


...

Done evaluating on DTD. Accuracy: 96.38%
Evaluate SUN397
Single task model
Classification head for ViT-L-14 on SUN397 exists at /data2/david3684/2024_arithmetic/checkpoints/ViT-L-14/head_SUN397_openai.pt
Loading classification head from /data2/david3684/2024_arithmetic/checkpoints/ViT-L-14/head_SUN397_openai.pt
Number of classes: 397
Train dataset size: 87003
Test dataset size: 21751


100%|██████████| 170/170 [04:17<00:00,  1.51s/it]


Done evaluating on SUN397. Accuracy: 82.64%
Multi task model
Classification head for ViT-L-14 on SUN397 exists at /data2/david3684/2024_arithmetic/checkpoints/ViT-L-14/head_SUN397_openai.pt
Loading classification head from /data2/david3684/2024_arithmetic/checkpoints/ViT-L-14/head_SUN397_openai.pt
Number of classes: 397
Train dataset size: 87003
Test dataset size: 21751


100%|██████████| 170/170 [04:15<00:00,  1.50s/it]

Done evaluating on SUN397. Accuracy: 81.04%


In [13]:
# _, _, val_preprocess = open_clip.create_model_and_transforms(
#             args.model, pretrained='openai', cache_dir=args.openclip_cachedir)
# dataset_1 = get_dataset(
#         args.tasks[0],
#         val_preprocess,
#         location=args.data_location,
#         batch_size=args.batch_size,
#         num_workers=16,
#         num_test_samples=None,
#     )
# dataloader_1 = get_dataloader(
#     dataset_1, is_train=False, args=args, image_encoder=None)

# dataset_2 = get_dataset(
#         args.tasks[1],
#         val_preprocess,
#         location=args.data_location,
#         batch_size=args.batch_size,
#         num_workers=16,
#         num_test_samples=args.num_test_samples,
#     )
# dataloader_2 = get_dataloader(
#     dataset_2, is_train=False, args=args, image_encoder=None)

Number of classes: 47
Train dataset size: 1880
Test dataset size: 1880
Number of classes: 397
Train dataset size: 87003
Test dataset size: 2048
